# Content Analysis

In [2]:
import urllib
import urllib.request
import requests
from bs4 import BeautifulSoup
from collections import deque
import csv
from textblob import TextBlob
import nltk
nltk.data.path.append("/Users/niklasstoehr/Libraries");

import json
import sys
import re

import networkx as nx
import matplotlib.pyplot as plt

## Topic Analysis

In [3]:
def topic_analysis(url, topic_dict):
                
    # Start Page Analsis_________________________________________________________________
    analysis_results = dict()
    
    try:  ## Webpage crawlable
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html)
            
    
        # Text Extraction _______________________________________________________________________

        # kill all script and style elements
        for script in soup(["script", "style", "head", "title", "[document]"]):
            script.extract()    # rip it out



        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        ## convert to lower case
        text = text.lower()
        ##exclude all text shorter than "min_characters""
        content = ""
        min_characters = 4
        for line in iter(text.splitlines()):
            if len(line) > min_characters:
                content = content + line + "\n"            


        # Check Amount of Videos _____________________________________________________________
        videos = len(soup.find_all('video', recursive=True))
        analysis_results["videos"] = videos



        # Check Connections to Social Media
        if "facebook" in content or "youtube" in content or "instagram" in content or "linkedin" in content or "twitter" in content:
            analysis_results["social_media"] = 1
        else:
            analysis_results["social_media"] = 0



        # Text Analysis _______________________________________________________________________

        for topic in topic_dict.keys():

            keyword_count = 0
            keywords = topic_dict[topic].split(",")

            for k in keywords:
                keyword_count += sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(k), content))

            analysis_results[topic] = keyword_count


        # Sentiment Analysis _______________________________________________________________________
        blob = TextBlob(content)
        sentiment = round(blob.sentiment.polarity,4)
        analysis_results["sentiment"] = sentiment


        # Language Analysis _______________________________________________________________________  
        language = blob.detect_language()
        analysis_results["language"] = language


        #print(analysis_results)

        return analysis_results
    
    
    
    except: ## Webpage not crawlable
        
        print("could not crawl webpage")
        analysis_results["emobility"] = 0
        analysis_results["mobility"] = 0
        analysis_results["ai"] = 0
        analysis_results["sentiment"] = 0.0
        analysis_results["language"] = "en"
        analysis_results["videos"] = 0
        analysis_results["social_media"] = 0

        if "facebook" in url or "youtube" in url or "instagram" in url or "linkedin" in url or "twitter" in url:
            analysis_results["social_media"] = 1

        return analysis_results # 'exit' function and return to caller


#webpage = "https://www.audi.com/en.html"
#topic_results = topic_analysis(webpage, topic_dict)

## Define Topic

In [4]:
topic_dict = dict()
topic_dict["emobility"] = "hybrid,battery,environment,electric"
topic_dict["mobility"] = "mobility,cloud,connectivity"
topic_dict["ai"] = "autonomous,intelligent,algorithm,intelligence"  

## Perform Analysis

In [9]:
domain_name = "toyota"

In [10]:
## Reader
nodes_csv = open('results/' + domain_name + '_nodes.csv')
#nodes_csv = open('results/' + domain_name + '_nodes.csv')

reader_count = csv.DictReader(nodes_csv)
## count length of csv file
row_count = sum(1 for row in reader_count) - 1  #header 
nodes_csv.seek(0) # go back to beginning of file


reader = csv.DictReader(nodes_csv)

#headers
header_list = ["id","label"] + list(next(reader).keys()) + ["videos","social media","emobility","mobility","ai","sentiment","language"]
header_string = ','.join(map(str, header_list)) 

## Writer
nodes_tags_csv = open('results/' + domain_name + '_nodes_tags.csv', "w")
#nodes_tags_csv.write('id,label,videos,social_media,emobility,mobility,ai,sentiment,language' + '\n')
nodes_tags_csv.write(str(header_string) + '\n')


## Do Analysis
nodes_csv.seek(0) # go back to beginning of file
reader = csv.DictReader(nodes_csv)
i = 0

for row in reader:
    
    print("\n", i ,"of", row_count)
    webpage = row['Address']
    
    ## delete all commas in dict values
    for key in row.keys(): 
        row[key] = row[key].replace(",", "")
    
    ## make comma-separated string of row dict values
    row_string = ','.join(map(str, row.values()))
        
    topic_results = topic_analysis(webpage, topic_dict)
    print(topic_results)
    
    ## id, label, address, titel, ....., videos, social media, emobility .... sentiment, language
    nodes_tags_csv.write(str(webpage) + "," + str(webpage) + "," + row_string + "," + str(topic_results['videos']) + "," + str(topic_results['social_media']) + "," + str(topic_results['emobility']) + "," + str(topic_results['mobility']) + "," + str(topic_results['ai']) + "," + str(topic_results['sentiment']) + "," + str(topic_results['language']) + "\n")

    i += 1

print("\n\n analysis terminated")
    
nodes_csv.close()
nodes_tags_csv.close()


 0 of 2653
{'videos': 0, 'social_media': 1, 'emobility': 43, 'mobility': 2, 'ai': 0, 'sentiment': 0.1776, 'language': 'en'}

 1 of 2653
{'videos': 3, 'social_media': 0, 'emobility': 46, 'mobility': 4, 'ai': 1, 'sentiment': 0.133, 'language': 'en'}

 2 of 2653
{'videos': 15, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.1387, 'language': 'en'}

 3 of 2653
{'videos': 0, 'social_media': 1, 'emobility': 5, 'mobility': 5, 'ai': 0, 'sentiment': 0.2604, 'language': 'en'}

 4 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.0986, 'language': 'en'}

 5 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.0837, 'language': 'en'}

 6 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.0837, 'language': 'en'}

 7 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.0986, 'language': 'en'}



{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.0837, 'language': 'en'}

 66 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.0784, 'language': 'en'}

 67 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.0837, 'language': 'en'}

 68 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.1138, 'language': 'en'}

 69 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.107, 'language': 'en'}

 70 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.0837, 'language': 'en'}

 71 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.0784, 'language': 'en'}

 72 of 2653
{'videos': 2, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.0736, 'language': 'en'}

 73 o

{'videos': 0, 'social_media': 0, 'emobility': 15, 'mobility': 2, 'ai': 0, 'sentiment': 0.1288, 'language': 'en'}

 131 of 2653
{'videos': 3, 'social_media': 0, 'emobility': 18, 'mobility': 4, 'ai': 3, 'sentiment': 0.1441, 'language': 'en'}

 132 of 2653
{'videos': 1, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.1617, 'language': 'en'}

 133 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.0986, 'language': 'en'}

 134 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.0986, 'language': 'en'}

 135 of 2653
{'videos': 0, 'social_media': 1, 'emobility': 21, 'mobility': 2, 'ai': 0, 'sentiment': 0.0788, 'language': 'en'}

 136 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.0885, 'language': 'en'}

 137 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.0986, 'language': 'en'

could not crawl webpage
{'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': 0.0, 'language': 'en', 'videos': 0, 'social_media': 0}

 196 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 20, 'mobility': 1, 'ai': 0, 'sentiment': 0.1528, 'language': 'en'}

 197 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 198 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.108, 'language': 'en'}

 199 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 39, 'mobility': 5, 'ai': 0, 'sentiment': 0.1184, 'language': 'en'}

 200 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 201 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': -0.075, 'language': 'en'}

 202 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 18, 'mobility': 1, 'ai': 4, 'sentiment': 0.1344

{'videos': 0, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.0885, 'language': 'en'}

 261 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 18, 'mobility': 1, 'ai': 8, 'sentiment': 0.1762, 'language': 'en'}

 262 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.0972, 'language': 'en'}

 263 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 27, 'mobility': 1, 'ai': 1, 'sentiment': 0.1321, 'language': 'en'}

 264 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': -0.075, 'language': 'en'}

 265 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': -0.075, 'language': 'en'}

 266 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 267 of 2653
{'videos': 3, 'social_media': 0, 'emobility': 20, 'mobility': 2, 'ai': 0, 'sentiment': 0.1401, 'language': 'en'}


...

{'videos': 2, 'social_media': 0, 'emobility': 37, 'mobility': 1, 'ai': 0, 'sentiment': -0.2472, 'language': 'es'}

 2392 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 2393 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 25, 'mobility': 1, 'ai': 0, 'sentiment': -0.5383, 'language': 'es'}

 2394 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 18, 'mobility': 6, 'ai': 0, 'sentiment': 0.1156, 'language': 'en'}

 2395 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 17, 'mobility': 1, 'ai': 0, 'sentiment': -0.3338, 'language': 'es'}

 2396 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 2397 of 2653
{'videos': 2, 'social_media': 0, 'emobility': 37, 'mobility': 1, 'ai': 0, 'sentiment': -0.2472, 'language': 'es'}

 2398 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'lang

{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': 0.1235, 'language': 'en'}

 2456 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 25, 'mobility': 1, 'ai': 0, 'sentiment': 0.091, 'language': 'en'}

 2457 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.0837, 'language': 'en'}

 2458 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': 0.127, 'language': 'en'}

 2459 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.0837, 'language': 'en'}

 2460 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': 0.1235, 'language': 'en'}

 2461 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 16, 'mobility': 1, 'ai': 4, 'sentiment': 0.1467, 'language': 'en'}

 2462 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': 0.1235, 'language': 'en

{'videos': 0, 'social_media': 0, 'emobility': 14, 'mobility': 1, 'ai': 0, 'sentiment': 0.0837, 'language': 'en'}

 2521 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 17, 'mobility': 3, 'ai': 1, 'sentiment': 0.1108, 'language': 'en'}

 2522 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 6, 'mobility': 0, 'ai': 0, 'sentiment': 0.1235, 'language': 'en'}

 2523 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.3034, 'language': 'es'}

 2524 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': 0.1157, 'language': 'en'}

 2525 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 13, 'mobility': 1, 'ai': 0, 'sentiment': 0.0885, 'language': 'en'}

 2526 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.3222, 'language': 'es'}

 2527 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.091, 'language': 

{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.3222, 'language': 'es'}

 2585 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 5, 'mobility': 0, 'ai': 0, 'sentiment': 0.1749, 'language': 'en'}

 2586 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 17, 'mobility': 6, 'ai': 0, 'sentiment': 0.1415, 'language': 'en'}

 2587 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': 0.1235, 'language': 'en'}

 2588 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 2589 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 2590 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 2591 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language':

{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.0948, 'language': 'en'}

 2649 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.1007, 'language': 'en'}

 2650 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 0, 'mobility': 0, 'ai': 0, 'sentiment': 0.0833, 'language': 'en'}

 2651 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 9, 'mobility': 1, 'ai': 0, 'sentiment': -0.2798, 'language': 'es'}

 2652 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 21, 'mobility': 1, 'ai': 0, 'sentiment': 0.0948, 'language': 'en'}

 2653 of 2653
{'videos': 0, 'social_media': 0, 'emobility': 24, 'mobility': 1, 'ai': 0, 'sentiment': 0.0967, 'language': 'en'}


 analysis terminated
